# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902068


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:17,  2.69s/it]

Rendering models:  13%|█▎        | 4/31 [00:05<00:51,  1.91s/it]

Rendering models:  16%|█▌        | 5/31 [00:05<00:35,  1.37s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:35,  1.40s/it]

...

Rendering models:  84%|████████▍ | 26/31 [00:09<00:00,  5.81it/s]

Rendering models:  87%|████████▋ | 27/31 [00:10<00:00,  5.63it/s]

Rendering models:  94%|█████████▎| 29/31 [00:10<00:00,  6.35it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  6.16it/s]

equidad1                              0.059045
kayleebug2017                         0.002347
not-logged-in-8c0c44faf754d344b18a    0.023806
not-logged-in-20487e5808245554c355    0.000509
Juval                                 0.033427
Thedunkmasta                          0.001780
sinjinza1984                          0.000619
not-logged-in-ab1612bc18cd7aab85db    0.000580
not-logged-in-bf2b11a652c0e0ab208a    0.000756
not-logged-in-2ef7f191c8c973ce093e    0.000659
cedar-phoenix                         0.000706
ericmalani                            0.000768
pangeli5                              0.001911
aidenr                                0.000575
peakscience7                          0.096887
Lavadude                              0.090433
not-logged-in-15498ebaa693dd5a198d    0.000569
not-logged-in-8700ff8251e0704f745c    0.004553
not-logged-in-92531cfb2471bb613fa9    0.470161
not-logged-in-986357d841d09bf87b26    0.064712
bookert                               2.235676
barrowwright 

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())